<a href="https://colab.research.google.com/github/usma11dia0/web_scraping_on_colab/blob/main/yahoo_scraping_outdoor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.2fPCSwHg1Z/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.jbXA5MFq0g/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.b47NMRrpGY/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [25]:
#環境変数設定
TARGET = 'アウトドア'
TARGET_URL = 'https://shopping.yahoo.co.jp/category/2938/list?view=grid&sc_i=shp_pc_catelist_nrwcgt'

NUM_TO_FETCH = 200
STORES_PER_PAGE = 30
DRIVER_WAIT_TIME = 2
SCROLL_PAUSE_TIME = 3
MAX_SCROLL_ATTEMPTS = 5

# 各種ファイル保存先
ACCU_LIST_PATH = '/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_yahoo/accumulated_list_yahoo_outdoor.csv'
SUBMIT_FILE_PATH ='/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_yahoo/result_list_yahoo_outdoor_canu.xlsx'

#取得データ格納先
result_dict = { 
    '会社名(ストア情報)': [],
    '住所(ストア情報)': [],
    'ストア名(ストア情報)':[],
    '運営責任者(問い合わせ情報)':[],
    '住所(問い合わせ情報)':[],
    '電話番号(問い合わせ情報)':[],
    'メールアドレス(問い合わせ情報)':[],
    'URL':[],
}

In [26]:
#標準ライブラリ
import os
import math
import json
import re
import time

#サードパーティライブラリ
import pandas as pd
from google.colab import drive
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from logging import (
    getLogger, 
    StreamHandler, 
    DEBUG, 
    INFO, 
    Formatter, 
    config,
)
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
#関数定義
def add_store_url(elements, store_urls, store_names, count_skip):
    for i, element in enumerate(elements, start = 0):
        if i > 29:
          break
        store_name = element.text
        store_url = element.get_attribute('href')
        if store_name not in accu_set and store_name not in store_names:
            store_urls.append(store_url)
            store_names.append(store_name)
        else:
            count_skip += 1
    return store_urls, store_names, count_skip

#スクロールしてページを更新
def scroll_page(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    attempts = 0
    while attempts < MAX_SCROLL_ATTEMPTS:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        attempts += 1

def is_japanese_phone_number(phone_number):
    # 固定電話のパターン
    landline_pattern = r'^0\d{1,4}-\d{1,4}-\d{3,4}$'
    # 携帯電話のパターン
    mobile_pattern = r'^0[789]0-\d{4}-\d{4}$'

    return bool(re.match(landline_pattern, phone_number) or re.match(mobile_pattern, phone_number))

In [31]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--ignore-certificate-errors')
options.add_argument("--disable-extensions")
options.add_argument("--disable-popup-blocking")
options.add_argument(
       "user-agent=Mozilla/5.0 (X11; Linux x86_64; rv:93.0) Gecko/20100101 Firefox/93.0"
    )
options.add_experimental_option("prefs", {
    "profile.managed_default_content_settings.images": 2,  # 画像の無効化
    "profile.managed_default_content_settings.plugins": 2,  # プラグインの無効化
})

driver = webdriver.Chrome('chromedriver', options=options)
driver.implicitly_wait(DRIVER_WAIT_TIME)

In [29]:
#ロガーの設定
with open('/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_yahoo/logging_config.json', 'r') as f:
    logger_config = json.load(f)
config.dictConfig(logger_config)
logger = getLogger('main')

try:
      driver.get(TARGET_URL)
      logger.debug(f'{TARGET}：トップページへ移動しました')
except Exception as e:
      logger.error(f'{TARGET}：トップページへ移動出来ませんでした: [e]')
      raise

2023-05-18 00:48:18,903 [DEBUG] main: アウトドア：トップページへ移動しました


In [30]:
#店舗リストの全ページ数を導出
p_element = driver.find_element(By.CSS_SELECTOR,".SearchResultsDisplayOptions_SearchResultsDisplayOptions__count__WBsPf")
num_all_stores = int(p_element.text.replace(',','')[:-1])
num_all_pages = math.ceil(num_all_stores / STORES_PER_PAGE)


#積み上げリストから既存取得の店舗URLを抽出
try:
  df_accu = pd.read_csv(ACCU_LIST_PATH, usecols=[2])
  accu_list = df_accu.iloc[:, 0].values.tolist()
  accu_set = set(accu_list)
except FileNotFoundError:
  accu_set = set()

# 目標件数を満たすまでstore_urlを取得
try:
  store_urls = []
  store_names = []
  count = 0
  count_skip = 0
  while len(store_urls) < NUM_TO_FETCH:
    start = 30 * count if count != 0 else 0
    scroll_page(driver)
    div_elements = driver.find_elements(By.CSS_SELECTOR,".SearchResult_SearchResult__storeRow__tXYSC a")
    div_elements = div_elements[start:]
    #ページ末尾までクロールが終了した場合、次のページへ遷移
    if len(div_elements) == 0:
      try:
        page_elements = driver.find_elements(By.CSS_SELECTOR, ".Pager_Pager__lbzCH")
        page_element = page_elements[-1]
        a_element = page_element.find_element(By.CSS_SELECTOR, "div:nth-child(3) a")
        page_url = a_element.get_attribute('href')
        driver.get(page_url)
        scroll_page(driver)
        count = 0
        start = 0
        div_elements = driver.find_elements(By.CSS_SELECTOR,".SearchResult_SearchResult__storeRow__tXYSC a")
        div_elements = div_elements[start:]
        logger.debug(f'{TARGET}：次の店舗一覧ページへ遷移しました)')
        print(page_url)
      except:
        logger.debug(f'{TARGET}：次の店舗一覧ページへ遷移出来ませんでした)')
    store_urls, store_names, count_skip = add_store_url(div_elements, store_urls, store_names, count_skip)
    logger.debug(f'{TARGET}：店舗リスト取得 (取得数:{len(store_urls)}, 重複数：{count_skip})')
    count += 1
except KeyboardInterrupt:
  logger.info(f'{TARGET}：店舗URL取得を中断しました:{len(store_urls)}個') 
  print(store_urls)
  print(store_names)
  driver.quit()
  raise
except  Exception as e:
  logger.error(f'{TARGET}：店舗リストへ移動出来ませんでした: [{e}]')
  driver.quit()
  raise
logger.info(f'{TARGET}：店舗URL取得完了:{len(store_urls)}個')  
print(store_urls)
print(store_names)

2023-05-18 00:48:43,144 [DEBUG] main: アウトドア：店舗リスト取得 (取得数:0, 重複数：30)
2023-05-18 00:48:55,886 [DEBUG] main: アウトドア：店舗リスト取得 (取得数:0, 重複数：60)
2023-05-18 00:49:26,468 [DEBUG] main: アウトドア：店舗リスト取得 (取得数:1, 重複数：89)
2023-05-18 00:50:02,131 [DEBUG] main: アウトドア：店舗リスト取得 (取得数:2, 重複数：118)
2023-05-18 00:50:30,233 [DEBUG] main: アウトドア：店舗リスト取得 (取得数:2, 重複数：148)
2023-05-18 00:50:41,641 [DEBUG] main: アウトドア：店舗リスト取得 (取得数:2, 重複数：178)
2023-05-18 00:51:07,326 [DEBUG] main: アウトドア：店舗リスト取得 (取得数:3, 重複数：207)
2023-05-18 00:51:22,170 [DEBUG] main: アウトドア：店舗リスト取得 (取得数:3, 重複数：237)
2023-05-18 00:51:26,110 [DEBUG] main: アウトドア：店舗リスト取得 (取得数:4, 重複数：266)
2023-05-18 00:51:30,498 [DEBUG] main: アウトドア：店舗リスト取得 (取得数:4, 重複数：296)
2023-05-18 00:51:36,518 [DEBUG] main: アウトドア：店舗リスト取得 (取得数:5, 重複数：325)
2023-05-18 00:51:41,471 [DEBUG] main: アウトドア：店舗リスト取得 (取得数:5, 重複数：355)
2023-05-18 00:51:46,446 [DEBUG] main: アウトドア：店舗リスト取得 (取得数:5, 重複数：385)
2023-05-18 00:51:51,484 [DEBUG] main: アウトドア：店舗リスト取得 (取得数:5, 重複数：415)
2023-05-18 00:51:56,640 [DEBUG] main:

KeyboardInterrupt: ignored

In [32]:
page_num = 0
#店舗個別ページにて情報取得
for store_url in store_urls:   
    name_store = "-"
    address_store = "-"
    store_store = "-"
    name_contact = "-"
    address_contact = "-"
    tel_contact = "-"
    mail_contact = "-"
    try:
        driver.get(store_url)
        page_num += 1
        store_store = store_names[page_num - 1]
        logger.debug(f'{TARGET}：店舗ページ{page_num}へ移動しました')

        #店舗情報ページURLを取得
        info_urls = driver.find_elements(By.CSS_SELECTOR,".elInfoItemLink")
        if not info_urls:
          info_urls = driver.find_elements(By.CSS_SELECTOR,".styles_infoItemLink__m2ij_")
        info_url = info_urls[0].get_attribute('href')
        print(info_url)
        driver.get(info_url)
        logger.debug(f'{TARGET}：店舗情報ページ{page_num}へ移動しました')
        name_store = driver.find_element(By.CSS_SELECTOR, ".elSectionContent .elRow:nth-child(1) .elRowContent .elRowContentText").text
        address_store = driver.find_element(By.CSS_SELECTOR, ".elSectionContent .elRow:nth-child(3) .elRowContent .elRowContentText").text
        div_elements =  driver.find_elements(By.CSS_SELECTOR, ".mdInformationTable")
        div_element = div_elements[1]
        name_contact = div_element.find_element(By.CSS_SELECTOR, ".elRows .elRow:nth-child(1) .elRowContentText").text
        address_contact = div_element.find_element(By.CSS_SELECTOR, ".elRows .elRow:nth-child(3) .elRowContentText").text
        tel_contact = div_element.find_element(By.CSS_SELECTOR, ".elRows .elRow:nth-child(4) .elRowContentText").text
        mail_contact = div_element.find_element(By.CSS_SELECTOR, ".elRows .elRow:nth-child(5) .elRowContentText").text
        if is_japanese_phone_number(mail_contact):
          mail_contact = div_element.find_element(By.CSS_SELECTOR, ".elRows .elRow:nth-child(6) .elRowContentText").text
    except Exception as e:
        logger.error(f'{TARGET}：会社概要ページがありませんでした。: [{store_store, store_url}]:[{e}]')
    finally:
      result_dict['会社名(ストア情報)'].append(name_store)
      result_dict['住所(ストア情報)'].append(address_store)
      result_dict['ストア名(ストア情報)'].append(store_store)
      result_dict['運営責任者(問い合わせ情報)'].append(name_contact)
      result_dict['住所(問い合わせ情報)'].append(address_contact)
      result_dict['電話番号(問い合わせ情報)'].append(tel_contact)
      result_dict['メールアドレス(問い合わせ情報)'].append(mail_contact)
      result_dict['URL'].append(store_url)

df_result = pd.DataFrame(result_dict)
df_result

2023-05-18 02:34:39,528 [DEBUG] main: アウトドア：店舗ページ1へ移動しました
https://store.shopping.yahoo.co.jp/racss/info.html
2023-05-18 02:34:43,212 [DEBUG] main: アウトドア：店舗情報ページ1へ移動しました
2023-05-18 02:34:45,500 [DEBUG] main: アウトドア：店舗ページ2へ移動しました
https://store.shopping.yahoo.co.jp/tenmafitsworld/info.html
2023-05-18 02:34:48,918 [DEBUG] main: アウトドア：店舗情報ページ2へ移動しました
2023-05-18 02:34:52,823 [DEBUG] main: アウトドア：店舗ページ3へ移動しました
2023-05-18 02:34:57,360 [ERROR] main: アウトドア：会社概要ページがありませんでした。: [('ダイユーエイト.com', 'https://store.shopping.yahoo.co.jp/daiyu8/?sc_i=shp_pc_catelist_itemlist_shsrg_strnm')]:[list index out of range]
2023-05-18 02:34:59,966 [DEBUG] main: アウトドア：店舗ページ4へ移動しました
https://store.shopping.yahoo.co.jp/rainbow-1-shop/info.html
2023-05-18 02:35:04,100 [DEBUG] main: アウトドア：店舗情報ページ4へ移動しました
2023-05-18 02:35:06,068 [DEBUG] main: アウトドア：店舗ページ5へ移動しました
2023-05-18 02:35:10,159 [ERROR] main: アウトドア：会社概要ページがありませんでした。: [('inglewood Beauty', 'https://store.shopping.yahoo.co.jp/biget/?sc_i=shp_pc_catelist_itemlist_shsrg_

,会社名(ストア情報),住所(ストア情報),ストア名(ストア情報),運営責任者(問い合わせ情報),住所(問い合わせ情報),電話番号(問い合わせ情報),メールアドレス(問い合わせ情報),URL
0,株式会社ラックス,富山県富山市西長江1-12-1,Fungoal,北島 正大,富山県富山市西長江1-12-1,076-481-8080,yahoo@racss.com,https://ck.storematch.jp/rd?v=4.2&c=z&campaign...
1,天馬株式会社,東京都北区赤羽1-63-6,テンマ フィッツワールド,商品企画部 長崎勇太,東京都北区赤羽1丁目63番6号,03-3598-5651,nettenma2@tenmacorp.co.jp,https://ck.storematch.jp/rd?v=4.2&c=E&campaign...
2,-,-,ダイユーエイト.com,-,-,-,-,https://store.shopping.yahoo.co.jp/daiyu8/?sc_...
3,RIREHA,広島県広島市中区舟入南3-5-3舟入南3丁目アパート101,Rainbow-Shop,藤原 江梨子,広島県広島市中区舟入南3-5-3舟入南3丁目アパート101,090-8067-2937,take_it_easy_eri@yahoo.co.jp,https://store.shopping.yahoo.co.jp/rainbow-1-s...
4,-,-,inglewood Beauty,-,-,-,-,https://store.shopping.yahoo.co.jp/biget/?sc_i...
...,...,...,...,...,...,...,...,...
121,株式会社アシーズ,広島県尾道市向東町12416-1,登山アウトドアのアシーズブリッジ,井野口 満,岡山県岡山市北区横井上５７－１,086-294-6610,0862946610@ab084.com,https://store.shopping.yahoo.co.jp/asses/?sc_i...
122,有限会社インディーズ,徳島県徳島市川内町竹須賀160-1,インディーズヤフー店,加藤 良和,徳島県徳島市川内町竹須賀160-1,088-676-2402,yahoo@indies-mc.com,https://store.shopping.yahoo.co.jp/indies-mc/?...
123,株式会社ジュネシス,大阪府堺市南区三原台二丁5番41号,エビススリー,西田 和平,大阪府堺市東区石原町1-125-3,072-276-4140,toiawase.ebisu3@junesis.co.jp,https://store.shopping.yahoo.co.jp/ebisu3/?sc_...
124,株式会社三治商会,大阪府大阪市浪速区稲荷2-7-24,e-暮らしRあーる,後藤 秀直,大阪府大阪市浪速区稲荷2-7-24,06-6562-6701,info-ya@r-arl.com,https://store.shopping.yahoo.co.jp/kurashi-arl...


In [33]:
# 提出ファイル用出力
if not os.path.isfile(SUBMIT_FILE_PATH):
    df_result.to_excel(SUBMIT_FILE_PATH, header=True, index=False)
else:
  with pd.ExcelWriter(SUBMIT_FILE_PATH, mode='a', if_sheet_exists='new') as writer:
    df_result.to_excel(writer)

# 積み上げリスト保存用出力
df_result.to_csv(ACCU_LIST_PATH, mode='a', header=True, index=False)

In [34]:
# 積み上げリスト確認
df_accu = pd.read_csv(ACCU_LIST_PATH)
df_accu

,会社名(ストア情報),住所(ストア情報),ストア名(ストア情報),運営責任者(問い合わせ情報),住所(問い合わせ情報),電話番号(問い合わせ情報),メールアドレス(問い合わせ情報),URL
0,-,-,viva ヤフーショッピング店,-,-,-,-,https://store.shopping.yahoo.co.jp/vivaenterpl...
1,株式会社アイボリーライフ,兵庫県西宮市上甲子園四丁目8番9号,アイボリー製造所2,金子 達也,兵庫県西宮市上甲子園5丁目1-1,050-3592-2941,info@withyou-club.com,https://store.shopping.yahoo.co.jp/ivory-mp2/?...
2,-,-,Swa Suwa,-,-,-,-,https://store.shopping.yahoo.co.jp/swasuwa/?sc...
3,＆MAGIC,東京都狛江市駒井町3-7-15-101,三代目つるや洋品店,川口 樹,東京都狛江市駒井町3-7-15-101,050-3196-2220,tsuruya.the3rd@gmail.com,https://store.shopping.yahoo.co.jp/supermarket...
4,株式会社Pleasant.japan,東京都狛江市岩戸北1丁目10-6番地甲武ビル201,pleasant.japan,根本 洋一,東京都狛江市岩戸北1-10-6 甲武ビル201,03-5761-4839,info@pleasant-japan.com,https://store.shopping.yahoo.co.jp/pleasant-ja...
...,...,...,...,...,...,...,...,...
2133,株式会社アシーズ,広島県尾道市向東町12416-1,登山アウトドアのアシーズブリッジ,井野口 満,岡山県岡山市北区横井上５７－１,086-294-6610,0862946610@ab084.com,https://store.shopping.yahoo.co.jp/asses/?sc_i...
2134,有限会社インディーズ,徳島県徳島市川内町竹須賀160-1,インディーズヤフー店,加藤 良和,徳島県徳島市川内町竹須賀160-1,088-676-2402,yahoo@indies-mc.com,https://store.shopping.yahoo.co.jp/indies-mc/?...
2135,株式会社ジュネシス,大阪府堺市南区三原台二丁5番41号,エビススリー,西田 和平,大阪府堺市東区石原町1-125-3,072-276-4140,toiawase.ebisu3@junesis.co.jp,https://store.shopping.yahoo.co.jp/ebisu3/?sc_...
2136,株式会社三治商会,大阪府大阪市浪速区稲荷2-7-24,e-暮らしRあーる,後藤 秀直,大阪府大阪市浪速区稲荷2-7-24,06-6562-6701,info-ya@r-arl.com,https://store.shopping.yahoo.co.jp/kurashi-arl...
